# Animated time-series (from 1900): Civil Liberties Index — Major Powers

This chart animates the changes over time in Civil Rights for 6 major nations. Of special note:
- The unbelievable effect of the Nazi party on the decimation of civil rights in Germany in the 1930s and 1940s
- The massive decline of France under German occupation (1940-45)
- The increase in Russia during the early "Glasnost" years and the subsequent decline under Putin
- The recent decrease in China under Xi's more "hard-line" policies
- The significant decrease in India under the Modi government


In [9]:
import pandas as pd
import plotly.graph_objects as go

countries = [
    "United States",
    "China",
    "Russia",
    "India",
    "France",
    "Germany"
]

value_col = "Civil liberties index (best estimate, aggregate: average)"

df = pd.read_csv("human_rights_index.csv")

d = df[["Entity", "Year", value_col]].dropna()
d = d[(d["Year"] >= 1900) & (d["Entity"].isin(countries))]
d = d.groupby(["Entity", "Year"], as_index=False)[value_col].mean().sort_values(["Entity", "Year"])

years = sorted(d["Year"].unique())
if not years:
    raise ValueError("No rows after 1900 for the selected countries (check Entity spelling).")

xmin, xmax = years[0], years[-1]
ymin, ymax = float(d[value_col].min()), float(d[value_col].max())

print(f"Years: {xmin}–{xmax}")


Years: 1900–2023


In [ ]:
# Build an animated cumulative line chart (each frame shows data up to that year)

# initial traces: only up to first year
start = xmin
fig = go.Figure([
    go.Scatter(
        x=d[(d.Entity == c) & (d.Year <= start)].Year,
        y=d[(d.Entity == c) & (d.Year <= start)][value_col],
        mode="lines",
        name=c,
    )
    for c in countries
])

# frames
fig.frames = [
    go.Frame(
        name=str(y),
        data=[
            go.Scatter(
                x=d[(d.Entity == c) & (d.Year <= y)].Year,
                y=d[(d.Entity == c) & (d.Year <= y)][value_col],
                mode="lines",
                name=c,
            )
            for c in countries
        ],
    )
    for y in years
]

fig.update_layout(
    title="Civil Liberties Index over Time (1900+) — Major Powers",
    xaxis=dict(title="Year", range=[xmin, xmax]),  # FIX: lock full range so it doesn't stick at 1900–1901
    yaxis=dict(title=value_col, range=[ymin, ymax]),
    hovermode="x unified",
    updatemenus=[{
        "type": "buttons",
        "direction": "left",
        "x": 0,
        "y": 1.12,
        "buttons": [
            {
                "label": "Play",
                "method": "animate",
                "args": [None, {"frame": {"duration": 120, "redraw": True}, "transition": {"duration": 0}, "fromcurrent": True}],
            },
            {
                "label": "Pause",
                "method": "animate",
                "args": [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}],
            },
        ],
    }],
    sliders=[{
        "currentvalue": {"prefix": "Year: "},
        "steps": [
            {"method": "animate", "label": str(y), "args": [[str(y)], {"mode": "immediate", "frame": {"duration": 0, "redraw": True}, "transition": {"duration": 0}}]}
            for y in years
        ],
    }],
)

fig.show()
